#### 미리 한글화된 lexicon_kr 을 base로 추가로 추가해줄 sentiment word를 계산해준다.
#### naive 하게 일자별로 정리된 단어를 이용한 프로세스를 먼저 정의하고 전날 16시 ~ 당일 15:59분 기준으로 뉴스를 정리하여 다시 단어사전을 만듦

#### 크롤링 뉴스가 부족한 관계로 2020 4월 11일 ~ 9월 22일 까지의 데이터만 대상으로 한다 

In [1]:
import datetime
from datetime import date

In [2]:
today = datetime.date.today()
today.weekday() # 0 = 월요일 , 1 = 화요일 , 2 = 수요일 , 3 = 목요일 , 4 = 금요일 , 5 = 토요일 , 6 = 일요일

0

In [3]:
date = date(2020, 9, 19)
date.weekday()

5

In [4]:
day_delta = datetime.timedelta(days=1) 
result = (date + day_delta).weekday() # date에 1을 더한 결과 갑이 나온다.
print(result)
print(type(result))

6
<class 'int'>


In [5]:
# date_int 가 평일 일때만  date_int를 프린트한다. 
date_int = result
if date_int < 5 :
    print(date_int)

In [6]:
start_date = datetime.date(2020, 4, 11)
end_date = datetime.date(2020, 9, 22)
length = (end_date - start_date).days

In [7]:
# 평일 데이터만 가져오기 위해 위에서 짠 것들을 합쳐준다.
date_weekday = [] # 평일날짜를 넣기위한 리스트 
for i in range(length+1) : # 마지막날을 포함하기 위해 length에 + 1 을 해준다. 
    date = start_date + i*day_delta # date 는 for 문을 돌며 day_delta로 1일 씩 더해지게 된다. 
    date_w = date.weekday() # date에 저장된 날짜를 weekday로 본다 0~6의 값을 가진다. 
    if date_w < 5 : # 조건문을 걸어 0~4 월~금 만 date_weekday 리스트에 추가하도록한다. 
        date_weekday.append(str(date))
# date_weekday # 평일만 포함된 date weekday 리스트가 나온다. 이를 이용해 평일 엑셀 데이터만 불러 오도록 한다. 

#### 주가 등락률을 기반으로 단어의 극성을 분류해 낸다 . 
#### 종목 토론실 공감 20이상 글을 가져올 수 있으면 좋은 자료가 될 것 같다 .

In [178]:
import pandas as pd
df = pd.read_csv('./data_stock/YG_1y.csv')
df

,년/월/일,종가,대비,거래량(주),거래대금(원),시가,고가,저가,시가총액(백만),상장주식수(주),등락률
0,2020-09-29,"58,000","1,000","758,762","43,007,076,200","57,700","58,000","54,900","1,066,451","18,387,079",0.017544
1,2020-09-28,"57,000","6,100","1,067,907","59,008,072,000","52,300","57,500","51,800","1,048,064","18,387,079",0.119843
2,2020-09-25,"50,900","-1,000","619,816","31,528,068,550","52,600","52,900","49,400","935,902","18,387,079",-0.019268
3,2020-09-24,"51,900","-4,000","767,893","41,149,411,700","54,500","55,600","51,800","954,289","18,387,079",-0.071556
4,2020-09-23,"55,900",700,"430,881","23,448,852,900","56,400","56,400","52,500","1,027,838","18,387,079",0.012681
...,...,...,...,...,...,...,...,...,...,...,...
245,2019-10-07,"23,400",550,"194,442","4,550,176,200","23,000","23,700","22,850","425,824","18,197,609",0.024070
246,2019-10-04,"22,850",-350,"181,631","4,125,474,550","23,250","23,250","22,450","415,815","18,197,609",-0.015086
247,2019-10-02,"23,200","-1,000","235,928","5,523,800,750","23,600","23,850","23,200","422,185","18,197,609",-0.041322
248,2019-10-01,"24,200","1,150","215,461","5,099,266,400","23,200","24,200","22,800","440,382","18,197,609",0.049892


In [179]:
df
df.columns = ['date', 'closing', 'diff', 'volume', 'transaction', 'market', 'high', 'low', 'capital', 'stocks', 'rof']
# date 날짜 , closing 종가 , diff 전일대비 , volume 거래량 , transaction 거래대금 , market 시가, high 고가, low 저가 , capital 시가총액 , stocks 상장주식수 , rof 등락률 

In [180]:
dates = []

df_date = df.iloc[:,:1]
df_date = df_date.sort_values(by='date',ascending=False)

list1 = df_date.values.tolist()

for i in range(len(df_date)):
    tokens = list1[i][0]
    token = tokens.split('/')
    year = token[0]
    month = token[1]
    #month = month.replace('0','')
    day = token[2]
    date = year + '-' + month + '-' + day 
    dates.append(date)
dates

IndexError: list index out of range

In [181]:
df['date'] = dates

ValueError: Length of values (0) does not match length of index (250)

In [182]:
df.loc[:,['date','diff']].values # 일자별 등략률을 가지고온다.

array([['2020-09-29', '1,000'],
       ['2020-09-28', '6,100'],
       ['2020-09-25', '-1,000'],
       ['2020-09-24', '-4,000'],
       ['2020-09-23', '700'],
       ['2020-09-22', '-400'],
       ['2020-09-21', '-2,400'],
       ['2020-09-18', '1,700'],
       ['2020-09-17', '-2,000'],
       ['2020-09-16', '300'],
       ['2020-09-15', '0'],
       ['2020-09-14', '-700'],
       ['2020-09-11', '1,000'],
       ['2020-09-10', '0'],
       ['2020-09-09', '100'],
       ['2020-09-08', '2,900'],
       ['2020-09-07', '2,400'],
       ['2020-09-04', '100'],
       ['2020-09-03', '2,000'],
       ['2020-09-02', '1,750'],
       ['2020-09-01', '1,800'],
       ['2020-08-31', '-2,350'],
       ['2020-08-28', '-1,900'],
       ['2020-08-27', '500'],
       ['2020-08-26', '1,700'],
       ['2020-08-25', '1,900'],
       ['2020-08-24', '100'],
       ['2020-08-21', '200'],
       ['2020-08-20', '-2,800'],
       ['2020-08-19', '2,550'],
       ['2020-08-18', '-1,750'],
       ['2020-08-14', '2

#### 뉴스 데이터 정리 

In [12]:
df1 = pd.read_excel('./YG_DATA/YG_DATAyg 2020-04-11.xlsx')
df1['title'].values.tolist()[5].split(']')[1]

" 정치권 표적된 음원 사재기, 선거철 '반짝 이슈'여선 안 되는 이유"

In [13]:
sen = df1['title'].values.tolist()[5]
sen = sen.split(']')
sen = sen[1]
sen = sen.strip()
sen

"정치권 표적된 음원 사재기, 선거철 '반짝 이슈'여선 안 되는 이유"

In [14]:
# 특수문자를 제거. 
import re 
text = sen
pattern = '[^\w\s]'
repl = ''
re.sub(pattern," ",text)

'정치권 표적된 음원 사재기  선거철  반짝 이슈 여선 안 되는 이유'

#### 위에것을 합쳐서 만들어본다 

In [15]:
#######################
sentences = [] 
list1 = list(set(df1['title'].values.tolist())) # 중복을 제거 한다.
pattern = '[^\w\s]'
repl = ''
for i in list1 :
    sen = i.split(']')
    if len(sen) >= 2 : # 스플릿 했을때 ] 가 있으면 len이 2 가 나오고 없으면 1이 나옴. 그중 문장을 선택하게 조건문을 부여
        sen = sen[1]
    else :
        sen = sen[0]
    sen = re.sub(pattern," ",sen) # pattern에서 정의한 특수문자 정규표현식 을 제거 
    sen = sen.strip() # 양옆의 공백을 제거 

    sentences.append(sen)
sentences

['블랙핑크  코첼라 20주년 다큐멘터리 출연  글로벌 위상',
 'K팝스타3  장한나  YG 떠나 홀로서기    데뷔 임박',
 'BTS 기생충 K 드라마  한국은 군대 없이 지구를 침략했다',
 '서원진  더블랙레이블 떠나 하트엔터行   쿠시와 한배',
 '위너   Remember  라이브영상 공개   흔들림 없는 가창력',
 '블랙핑크  코첼라 20주년 다큐 출연  압도적 영향력',
 '위너  Remember  라이브 클립 공개  믿고 듣는 명성 입증',
 '위너   Remember  라이브 영상 흔들림 없는 가창력',
 '정치권 표적된 음원 사재기  선거철  반짝 이슈 여선 안 되는 이유',
 '위너   Remember  라이브 영상 공개 CD 삼킨 가창력',
 'Quibi  9 Shows Worth Your Time  Or at Least a Few Minutes',
 '블랙핑크  美코첼라 20주년 다큐 출연  글로벌 위상',
 'Inside the Strip Clubs of Instagram',
 '블랙핑크  코첼라 20주년 다큐 출연  글로벌 위상',
 '케이씨씨글라스  한화생명  남양유업우  삼성바',
 '그룹 위너   Remember  라이브 영상 화제']

#### set -> list 를 사용해서 중복을 제거하였다.
#### 이후 KOMORAN 형태소 분석기를 이용하여 TF를 세어주고, 등락률을 계산하여 Word Score를 만들어준다.


In [16]:
def replaceAllconma(df, start, finish): # start, finish 열의 인덱스를 지정해 주면된다. 
    for j in range(start, finish+1):
        for i in range(len(df.iloc[:,j])):
            df.iloc[:,j][i] = df.iloc[:,j][i].replace(",", "")
    return df

In [17]:
df = replaceAllconma(df, 1, 9)

C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [18]:
# 등락률 : 전일 종가 대비 얼마나 변화가 있었는지. = > rate of fluctuation ROF 현재 df 에서는 전일종가 / 당일대비 
df.loc[:,['date', 'closing', 'diff']]
# 전일종가 
closing = df.iloc[1].iloc[1]  # 9월 28일 종가
print(closing)
# 당일 대비
diff = df.iloc[0].iloc[2] # 당일 대비
print(diff)
print(type(closing))
print(type(diff))
# 등락률
rof = int(diff)/int(closing)
print(rof)

57000
1000
<class 'str'>
<class 'str'>
0.017543859649122806


#### 만들다가 엑셀로 자료 정리하여 rof 를 간단하게 해결 이후 샘플로 만든 9월 19일 tf를 계산해 word score를 계산해본다.

In [127]:
# 전일 종가 데이터가 없는 마지막 열을 삭제
df = df.dropna()
df = df.loc[:,['date', 'rof']]

#### user dic에 한글자 만 있는 경우가 발생 이를 삭제해 주어야함 

In [ ]:
with open('userdicCanSur_YG_Sentiment.txt', 'r' ) as f :
    line = f.readlines()
    print(line)
    
# 개행문자를 포함하여 글자수가 2개 이하인 모든 글자를 삭제. 
for i in line :
    i.strip()
    if len(i) < 3 :
        line.remove(i)
 
with open('userdicCanSur_YG_Sentiment.txt', 'w') as f:
    for i in line:
        f.write(i)

# TF - Score계산

In [221]:
date = '2020-04-13'

df1 = pd.read_excel(f'./data_news/YG_DATAyg {date}.xlsx')
sentences = [] 
list1 = list(set(df1['title'].values.tolist())) # 중복을 제거 한다.
pattern = '[^\w\s]'
repl = ''
for i in list1 :
    sen = i.split(']')
    if len(sen) >= 2 : # 스플릿 했을때 ] 가 있으면 len이 2 가 나오고 없으면 1이 나옴. 그중 문장을 선택하게 조건문을 부여
        sen = sen[1]
    else :
        sen = sen[0]
    sen = re.sub(pattern," ",sen) # pattern에서 정의한 특수문자 정규표현식 을 제거 
    sen = sen.strip() # 양옆의 공백을 제거 

    sentences.append(sen)
sentences

['AI 시스템 포착을 통해 오늘의 핫한 종목   삼화네트웍스  초',
 '플랙  송민호 강승윤 컬렉션 판매 호조',
 '박진영  GOT7 신곡  NOT BY THE MOON  작사 작곡 편곡 참여 무한 사랑 메시',
 '위너 이승훈도 입대 16일 훈련소 입소',
 'GOT7  20일 컴백 쇼케이스서  DYE   NOT BY THE MOON  무대 최초공개',
 '헤드업때 축 무너지면 바닥에서 헤엄치세요',
 '위너 이승훈  오는 16일 입대  건강히 돌아오겠다',
 '갓세븐  20일 컴백 쇼케이스서  NOT BY THE MOON  무대 최초공개',
 'YG PLUS   17 20  상승폭 확대',
 '위너  이승훈 입소 앞두고 신개념 온라인 사인회  팬들과 잊지 못할 추억 공식',
 '코스피 1 88  하락하며 1825 76에 마감',
 'boy band member military duty',
 '위너 이승훈  16일 논산훈련소 입소   사회복무요원 대체복무',
 'YG PLUS   17 60  VI 발동',
 'YG PLUS  15  이상 상승  주가 상승 중  단기간 골든크로스 형성',
 '위너  신보 Remember 온라인 팬사인회 성료  참신한 시도 속 진지한 팬사랑',
 '싸이 대기록 67억뷰   강남스타일  35억뷰  젠틀맨  13억뷰 등 견인차',
 'YG PLUS  코오롱인더  LS네트웍스  윌비스  신송',
 '위너 이승훈  16일 입소 사회복무요원 대체복무  팬들 향한 손편지 공개',
 'YG PLUS  와이지 음반 음원 전담으로 고성장 기대',
 'GOT7 신곡  낫 바이 더 문    박진영 작사 작곡 편곡 참여',
 '더 캡쳐   젝키 오락관   셧 아이  시즌1  타이거 킹',
 '13일  코스피 개인 순매수 외국인 기관 매도  코스닥 개인 순매',
 'GOT7  컴백 쇼케이스 개최   타이틀곡  NOT BY THE MOON  무대 공개',
 '13일 코스피 주가 Top 50 상승률',
 '위너 이승훈   16일 입대  발표  몸 건

In [232]:
from konlpy.tag import Komoran
import pandas as pd
komoran_userdic=Komoran(userdic='./userdicCanSur_YG_Sentiment.txt')
corpus = sentences
# nouns : 명사 추출
# morphs : 형태소 추출
# pos : 품사 부착
wordCount = {}
wordlist = []
answer = []
for i in corpus:
    wordlist.append(komoran_userdic.nouns(i))
for i in wordlist:
    answer += i
print(answer)

['시스템', '포착', '오늘', '핫', '종목', '삼화네트웍스', '초', '송민호', '강승윤', '컬렉션', '판매', '호조', '박진영', 'GOT7', '신곡', '작사', '작곡', '편곡', '참여', '무한', '사랑', '메시', '위너', '이승훈', '입대', '일', '훈련소', '입소', 'GOT7', '일', '컴백', '쇼케이스', '무대', '최초', '공개', '헤드', '업', '때', '바닥', '위너', '이승훈', '일', '입대', '갓', '세븐', '일', '컴백', '쇼케이스', '무대', '최초', '공개', 'YG', '상승', '폭', '확대', '위너', '이승훈', '입소', '개념', '온라인', '사인회', '팬', '추억', '공식', '코스', '피', '하락', '마감', '위너', '이승훈', '일', '논산훈련소', '입소', '사회복무요원', '대체', '복무', 'YG', '발동', 'YG', '이상', '상승', '주가', '상승', '중', '단기간', '골든크로스', '형성', '위너', '보', '온라인', '팬', '사인회', '시도', '속', '팬', '사랑', '싸이', '기록', '강남스타일', '젠틀맨', '등', '견인차', 'YG', '코오롱', '인', 'LS네트웍스', '송', '위너', '이승훈', '일', '입소', '사회복무요원', '대체', '복무', '팬', '손', '편지', '공개', 'YG', '음반', '음원', '전담', '고성', '장', '기대', 'GOT7', '신곡', '낫', '바', '이', '더 문', '박진영', '작사', '작곡', '편곡', '참여', '더 캡', '오락', '아이', '시즌', '타이거', '킹', '일', '코스', '피', '개인', '순', '매수', '외국인', '기관', '매도', '코스닥', '개인', '매', 'GOT7', '컴백', '쇼케이스', '개최', '타이틀', '곡', '무대', '공

In [233]:
# 빈도수가 2번 이상인 단어만을 대상으로 한다. 
from collections import Counter
attention = {}
attention = Counter(answer)
df1 = pd.DataFrame(attention.keys())
df1.columns =['Name']
df1['Frequency'] = pd.DataFrame(attention.values())
df1 = df1[df1['Frequency'] >= 2] 
df1.reset_index(inplace = True)
df1 = df1.drop('index', axis =1)
df1

,Name,Frequency
0,종목,3
1,박진영,5
2,GOT7,7
3,신곡,5
4,작사,4
...,...,...
65,댄스,2
66,퍼포먼스,4
67,김진우,2
68,월,2


In [234]:
# 한글자 데이터를 삭제해 준다. 
for i in range(len(df1)):
    word = df1['Name'][i] # iloc로 하면 df를 drop 했을때 키를 못찾는 에러가 발생함. 
    print(i, word)
    if len(word) < 2 :
        print(i, word)
        df1.drop(index=i, inplace=True)
df1

0 종목
1 박진영
2 GOT7
3 신곡
4 작사
5 작곡
6 편곡
7 참여
8 사랑
9 위너
10 이승훈
11 입대
12 일
12 일
13 훈련소
14 입소
15 컴백
16 쇼케이스
17 무대
18 최초
19 공개
20 갓
20 갓
21 세븐
22 YG
23 상승
24 개념
25 온라인
26 사인회
27 팬
27 팬
28 코스
29 피
29 피
30 하락
31 마감
32 논산훈련소
33 사회복무요원
34 대체
35 복무
36 주가
37 시도
38 등
38 등
39 개인
40 개최
41 타이틀
42 곡
42 곡
43 몸
43 몸
44 엔터
45 방탄소년단
46 씨
46 씨
47 푸드
48 회
48 회
49 전율
50 위
50 위
51 화면
52 해설
53 방송
54 편성
55 안내
56 대
56 대
57 최근
58 논산
59 최선
60 트레
61 지훈
62 도
62 도
63 영
63 영
64 소정환
65 댄스
66 퍼포먼스
67 김진우
68 월
68 월
69 급등


,Name,Frequency
0,종목,3
1,박진영,5
2,GOT7,7
3,신곡,5
4,작사,4
5,작곡,4
6,편곡,4
7,참여,5
8,사랑,2
9,위너,14


In [235]:
df = df[ df['date'] == date ]
rof = df['rof'].iloc[0]
rof

-0.031141869

In [236]:
df1['tf_rof'] = df1['Frequency']*rof

In [237]:
df1

,Name,Frequency,tf_rof
0,종목,3,-0.093426
1,박진영,5,-0.155709
2,GOT7,7,-0.217993
3,신곡,5,-0.155709
4,작사,4,-0.124567
5,작곡,4,-0.124567
6,편곡,4,-0.124567
7,참여,5,-0.155709
8,사랑,2,-0.062284
9,위너,14,-0.435986
